# Depiction of violation of yaw and pitch constraint

In [25]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d.proj3d import proj_transform
from math import cos as cos
from math import sin as sin

deg2rad = lambda x: x * np.pi / 180

LONG = [
  [
    [200, 500, 200, deg2rad(180), deg2rad(-5)],
    [500, 350, 100, deg2rad(0), deg2rad(-5)],
    [467.70, 449.56],
    "Long 1"
  ],[
    [100, -400, 100, deg2rad(30), deg2rad(0)],
    [500, -700, 0, deg2rad(150), deg2rad(0)],
    [649.52, 636.12],
    "Long 2"
  ],[
    [-200, 200, 250, deg2rad(240), deg2rad(15)],
    [ 500, 800,   0, deg2rad(45), deg2rad(15)],
    [1088.10, 1063.41],
    "Long 3"
  ],[
    [-300, 1200, 350, deg2rad(160), deg2rad(0)],
    [1000,  200,   0, deg2rad(30), deg2rad(0)],
    [1802.60, 1789.21],
    "Long 4"
  ],[
    [-500, -300, 600, deg2rad(150), deg2rad(10)],
    [1200,  900, 100, deg2rad(300), deg2rad(10)],
    [2245.14, 2216.40],
    "Long 5"
  ]
]

SHORT = [
  [
    [120, -30, 250, deg2rad(100), deg2rad(-10)],
    [220, 150, 100, deg2rad(300), deg2rad(-10)],
    [588.60, 583.47],
    "Short 1"
  ],[
    [380, 230, 200, deg2rad(30), deg2rad(0)],
    [280, 150,  30, deg2rad(200), deg2rad(0)],
    [667.71, 658.53],
    "Short 2"
  ],[
    [-80, 10, 250, deg2rad(20), deg2rad(0)],
    [ 50, 70,   0, deg2rad(240), deg2rad(0)],
    [979.34, 968.25],
    "Short 3"
  ],[
    [400, -250, 600, deg2rad(350), deg2rad(0)],
    [600, -150, 300, deg2rad(150), deg2rad(0)],
    [1169.73, 1161.55],
    "Short 4"
  ],[
    [-200, -200, 450, deg2rad(340), deg2rad(0)],
    [-300,  -80, 100, deg2rad(100), deg2rad(0)],
    [1367.56, 1354.12],
    "Short 5"
  ]
]

ADDITIONAL = [
    [
        [120, 40, 20, deg2rad(90), deg2rad(-5)],
        [300, 40, 15, deg2rad(-90), deg2rad(-5)],
        [],
        "Additional 1"
    ],
    [
        [120, 40, 20, deg2rad(90), deg2rad(-15)],
        [130, 120, 41, deg2rad(85), deg2rad(20)],
        [],
        "Additional 2"
    ],
    [
        [0, 0, 0, deg2rad(30), deg2rad(10)],
        [5, 10, 15, deg2rad(190), deg2rad(10)],
        [],
        "Additional 3"
    ],
    [
        [0, 0, 0, deg2rad(-30), deg2rad(10)],
        [0, -30, 15, deg2rad(190), deg2rad(10)],
        [],
        "Additional 4"
    ]
]

# We load the csv file containing the path
filename = "Additional 1"
# df = pd.read_csv(filename + ".csv")
df = pd.read_csv(filename + ".csv")

# Providing the length of the heading vector
length = 30.0

plt.rcParams['text.usetex'] = True

class Arrow3D(FancyArrowPatch):

    def __init__(self, x, y, z, dx, dy, dz, *args, **kwargs):
        super().__init__((0, 0), (0, 0), *args, **kwargs)
        self._xyz = (x, y, z)
        self._dxdydz = (dx, dy, dz)

    def draw(self, renderer):
        x1, y1, z1 = self._xyz
        dx, dy, dz = self._dxdydz
        x2, y2, z2 = (x1 + dx, y1 + dy, z1 + dz)

        xs, ys, zs = proj_transform((x1, x2), (y1, y2), (z1, z2), self.axes.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))
        super().draw(renderer)
        
    def do_3d_projection(self, renderer=None):
        x1, y1, z1 = self._xyz
        dx, dy, dz = self._dxdydz
        x2, y2, z2 = (x1 + dx, y1 + dy, z1 + dz)

        xs, ys, zs = proj_transform((x1, x2), (y1, y2), (z1, z2), self.axes.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))

        return np.min(zs)
    
def _arrow3D(ax, x, y, z, dx, dy, dz, *args, **kwargs):
    '''Add an 3d arrow to an `Axes3D` instance.'''

    arrow = Arrow3D(x, y, z, dx, dy, dz, *args, **kwargs)
    ax.add_artist(arrow)
    return arrow

setattr(Axes3D, 'arrow3D', _arrow3D)

In [26]:
%matplotlib qt
# We plot the 3D path
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
plt.tight_layout()

for data in ADDITIONAL:
    if data[-1] == filename:

        print('Found instance ', data[-1])
        break

# We plot the initial location and heading and final location and heading
ini_loc = np.array(data[0][:3])
fin_loc = np.array(data[1][:3])
ini_heading_angle = data[0][3]; fin_heading_angle = data[1][3]
ini_pitch_angle = data[0][4]; fin_pitch_angle = data[1][4]
ini_heading = np.array([cos(ini_heading_angle)*cos(ini_pitch_angle), sin(ini_heading_angle)*cos(ini_pitch_angle), sin(ini_pitch_angle)])
fin_heading = np.array([cos(fin_heading_angle)*cos(fin_pitch_angle), sin(fin_heading_angle)*cos(fin_pitch_angle), sin(fin_pitch_angle)])

# Plotting the initial and final location and heading
ax.scatter(ini_loc[0], ini_loc[1], ini_loc[2], linewidth = 2, marker = 'o', s = 50, color = 'red', label = 'Initial')
ax.arrow3D(ini_loc[0], ini_loc[1], ini_loc[2], length*ini_heading[0],\
                length*ini_heading[1], length*ini_heading[2], mutation_scale=20, fc='red')
ax.scatter(fin_loc[0], fin_loc[1], fin_loc[2], linewidth = 2, marker = 'D', s = 50, color = 'blue', label = 'Final')
ax.arrow3D(fin_loc[0], fin_loc[1], fin_loc[2], length*fin_heading[0],\
                length*fin_heading[1], length*fin_heading[2], mutation_scale=20, fc='blue')

# Plotting the path
ax.plot3D(df['X'], df['Y'], df['Z'], color = 'black', linewidth = 2, label = 'Path')

ax.legend(fontsize = 14, loc = 0, ncol = 3)

ax.tick_params(axis='both', which='major', labelsize = 14)
ax.tick_params(axis='both', which='minor', labelsize = 14)

ax.set_xlabel(r'$X$ (m)', fontsize = 18)
ax.set_ylabel(r'$Y$ (m)', fontsize = 18)
ax.set_zlabel(r'$Z$ (m)', fontsize = 18)

# We show the trajectory plot
plt.show()

Found instance  Additional 1


# Loading aircraft model

We load the aircraft model and show pitch rate and yaw rate constraints, and temporarily inaccessible regions due to the motion constraints

In [31]:
import copy
import math
import sys, os
import time

# We import the file for visualizing the aircraft
# Including the following command to ensure that python is able to find the relevant files afer changing directory
sys.path.insert(0, '')
# Obtaining the current directory
cwd = os.getcwd()
# current_directory = Path(__file__).parent
# path_str = str(current_directory)

# Importing code for plotting
# rel_path = '..\Visualization'
rel_path = os.path.join('..', 'Visualization')
abs_path = os.path.abspath(os.path.join(cwd, rel_path))
os.chdir(abs_path)
from view_manager import ViewManager
from msg_state import MsgState
from visualization_simulation import generate_points_sphere

# Returning to initial directory
os.chdir(cwd)

# We provide the pitch rate and yaw rate constraints
Rpitch = 40
Ryaw = 50
# Rpitch = 60
# Ryaw = 40

# We provide the scale of the UAV model
scale = 5
length = 30.0

# We also specify the roll angle for the vehicle
roll_angle = 0*math.pi/180

# We compute the lateral direction vector's direction cosines
lateral_dir = math.cos(roll_angle)*np.array([cos(ini_heading_angle + math.pi/2), sin(ini_heading_angle + math.pi/2), 0])\
      + math.sin(roll_angle)*np.cross(ini_heading, np.array([cos(ini_heading_angle + math.pi/2), sin(ini_heading_angle + math.pi/2), 0]))

# We compute the normal direction for the vehicle
normal_dir = np.cross(ini_heading, lateral_dir)

ini_config = np.array([ini_loc, ini_heading, lateral_dir, normal_dir])

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
plt.tight_layout()

# Make figure full screen
manager = plt.get_current_fig_manager()
manager.full_screen_toggle()

# We plot the initial location and heading and final location and heading
ini_loc = np.array(data[0][:3])
fin_loc = np.array(data[1][:3])
ini_heading_angle = data[0][3]; fin_heading_angle = data[1][3]
ini_pitch_angle = data[0][4]; fin_pitch_angle = data[1][4]
ini_heading = np.array([cos(ini_heading_angle)*cos(ini_pitch_angle), sin(ini_heading_angle)*cos(ini_pitch_angle), sin(ini_pitch_angle)])
fin_heading = np.array([cos(fin_heading_angle)*cos(fin_pitch_angle), sin(fin_heading_angle)*cos(fin_pitch_angle), sin(fin_pitch_angle)])

# Plotting the initial and final location and heading
ax.scatter(ini_loc[0], ini_loc[1], ini_loc[2], linewidth = 2, marker = 'o', s = 50, color = 'red', label = 'Initial')
ax.arrow3D(ini_loc[0], ini_loc[1], ini_loc[2], length*ini_heading[0],\
                length*ini_heading[1], length*ini_heading[2], mutation_scale=20, fc='red')
ax.scatter(fin_loc[0], fin_loc[1], fin_loc[2], linewidth = 2, marker = 'D', s = 50, color = 'blue', label = 'Final')
ax.arrow3D(fin_loc[0], fin_loc[1], fin_loc[2], length*fin_heading[0],\
                length*fin_heading[1], length*fin_heading[2], mutation_scale=20, fc='blue')

# Plotting the path
ax.plot3D(df['X'], df['Y'], df['Z'], color = 'black', linewidth = 2, label = 'Path')

# We plot the aircraft
true_state = MsgState()
true_state.north = ini_loc[0]
true_state.east = ini_loc[1]
true_state.altitude = -ini_loc[2]

true_state.theta = -ini_pitch_angle # NOTE: WE ARE DEFINING PITCH ANGLE IN THIS MANNER SINCE POSITIVE "PITCH ANGLE" IN
# AEROSPACE DEPICTS ELEVATION ANGLE, WHICH IS OPPOSITE TO POSITIVE ROTATION ABOUT Y AXIS
true_state.psi = ini_heading_angle
true_state.phi = roll_angle

viewers = ViewManager(ax, animation=True, video=False, scale_aircraft=scale)
viewers.update(fig, time.time(), true_state = true_state,  # true states
        )

# ax.legend(fontsize = 14, loc = 0, ncol = 3)
# ax.legend(fontsize = 14, loc = 'upper center', ncol = 3)
ax.legend(fontsize = 22, loc = 'upper center', ncol = 3)

# ax.tick_params(axis='both', which='major', labelsize = 14)
# ax.tick_params(axis='both', which='minor', labelsize = 14)
ax.tick_params(axis='both', which='major', labelsize = 22)
ax.tick_params(axis='both', which='minor', labelsize = 22)

# ax.set_xlabel(r'$X$ (m)', fontsize = 18)
# ax.set_ylabel(r'$Y$ (m)', fontsize = 18)
# ax.set_zlabel(r'$Z$ (m)', fontsize = 18)
ax.set_xlabel(r'$X$ (m)', fontsize = 28, labelpad = 40)
ax.set_ylabel(r'$Y$ (m)', fontsize = 28, labelpad = 20)
ax.set_zlabel(r'$Z$ (m)', fontsize = 28, labelpad = 20)

# ax.view_init(elev=0, azim=0)
# ax.view_init(elev=90, azim=0)

# ax.view_init(elev=39, azim=64)
# ax.view_init(elev=72, azim=58)
ax.view_init(elev=36, azim=16)
# ax.view_init(elev=44, azim=0)
# ax.view_init(elev=0, azim=90)

# ax.view_init(elev = 55, azim = -50)

# ax.view_init(elev = 20, azim = -158)

# ax.view_init(elev = 16, azim = -8)

# We plot the spheres around the vehicle
# Considering spheres for the pitch motion
# xfin, yfin, zfin = generate_points_sphere([ini_config[0, i] + Rpitch*ini_config[3, i] for i in range(3)], Rpitch)
# ax.plot_surface(xfin, yfin, zfin, color = 'orange', alpha=0.2)

# xfin, yfin, zfin = generate_points_sphere([ini_config[0, i] - Rpitch*ini_config[3, i] for i in range(3)], Rpitch)
# ax.plot_surface(xfin, yfin, zfin, color = 'magenta', alpha=0.2)

# Considering spheres for the yaw motion
xfin, yfin, zfin = generate_points_sphere([ini_config[0, i] + Ryaw*ini_config[2, i] for i in range(3)], Ryaw)
ax.plot_surface(xfin, yfin, zfin, color = 'green', alpha=0.2)

xfin, yfin, zfin = generate_points_sphere([ini_config[0, i] - Ryaw*ini_config[2, i] for i in range(3)], Ryaw)
ax.plot_surface(xfin, yfin, zfin, color = 'blue', alpha=0.2)

plt.axis('scaled')

# Saving the figure
plt.savefig('Yaw_rate_violation.png', dpi = 300)
# plt.savefig('Pitch_rate_violation.png', dpi = 300)
# plt.savefig('Vana_Additional_inside_top_sphere.png', dpi = 300)
# plt.savefig('Vana_Additional_inside_right_sphere.png', dpi = 300)

viewers.close()
plt.show()


In [16]:
# We check if any portion of the path lies inside one of the spheres
for i in range(len(df)):

    loc = np.array([df['X'][i], df['Y'][i], df['Z'][i]])

    if np.linalg.norm(loc - np.array([ini_config[0, i] + Rpitch*ini_config[3, i] for i in range(3)])) < 0.99*Rpitch:
        raise Exception('Inside inner sphere')
    elif np.linalg.norm(loc - np.array([ini_config[0, i] - Rpitch*ini_config[3, i] for i in range(3)])) < 0.99*Rpitch:
        raise Exception('Inside outer sphere')
    elif np.linalg.norm(loc - np.array([ini_config[0, i] + Ryaw*ini_config[2, i] for i in range(3)])) < 0.99*Ryaw:
        raise Exception('Inside left sphere')
    elif np.linalg.norm(loc - np.array([ini_config[0, i] - Ryaw*ini_config[2, i] for i in range(3)])) < 0.99*Ryaw:
        raise Exception('Inside right sphere')

Exception: Inside right sphere

We loop through the considered instances and try to find a roll angle that would violate the motion constraint.

In [ ]:
Rpitch = 40

# for data in ["Long 1.csv", "Long 2.csv", "Long 3.csv", "Long 4.csv", "Long 5.csv", "Short 1.csv", "Short 2.csv", "Short 3.csv", "Short 4.csv", "Short 5.csv"]:
for data in ["Additional 1.csv"]:
    
    print(data)
    df = pd.read_csv(data)
    ini_loc = np.array([df['X'].iloc[0], df['Y'].iloc[0], df['Z'].iloc[0]])
    fin_loc = np.array([df['X'].iloc[-1], df['Y'].iloc[-1], df['Z'].iloc[-1]])
    ini_heading_angle = df['Heading'].iloc[0]
    ini_pitch_angle = df['Pitch'].iloc[0]
    ini_heading = np.array([cos(df['Heading'].iloc[0])*cos(df['Pitch'].iloc[0]), sin(df['Heading'].iloc[0])*cos(df['Pitch'].iloc[0]), sin(df['Pitch'].iloc[0])])
    fin_heading = np.array([cos(df['Heading'].iloc[-1])*cos(df['Pitch'].iloc[-1]), sin(df['Heading'].iloc[-1])*cos(df['Pitch'].iloc[-1]), sin(df['Pitch'].iloc[-1])])

    # We also specify the roll angle for the vehicle
    for roll_angle in [-30*math.pi/180, -15*math.pi/180, 0*math.pi/180, 15*math.pi/180, 30*math.pi/180]:

        print('Roll angle is ', roll_angle)

        # We compute the lateral direction vector's direction cosines
        lateral_dir = math.cos(roll_angle)*np.array([cos(ini_heading_angle + math.pi/2), sin(ini_heading_angle + math.pi/2), 0])\
            + math.sin(roll_angle)*np.cross(ini_heading, np.array([cos(ini_heading_angle + math.pi/2), sin(ini_heading_angle + math.pi/2), 0]))

        # We compute the normal direction for the vehicle
        normal_dir = np.cross(ini_heading, lateral_dir)

        ini_config = np.array([ini_loc, ini_heading, lateral_dir, normal_dir])

        for Ryaw in [20, 30, 40, 50, 60]:

            print('Ryaw is ', Ryaw)

            # We check if any portion of the path lies inside one of the spheres
            for i in range(len(df)):

                loc = np.array([df['X'][i], df['Y'][i], df['Z'][i]])

                if np.linalg.norm(loc - np.array([ini_config[0, i] + Rpitch*ini_config[3, i] for i in range(3)])) < 0.99*Rpitch:
                    raise Exception('Inside inner sphere')
                elif np.linalg.norm(loc - np.array([ini_config[0, i] - Rpitch*ini_config[3, i] for i in range(3)])) < 0.99*Rpitch:
                    raise Exception('Inside outer sphere')
                elif np.linalg.norm(loc - np.array([ini_config[0, i] + Ryaw*ini_config[2, i] for i in range(3)])) < 0.99*Ryaw:
                    raise Exception('Inside left sphere')
                elif np.linalg.norm(loc - np.array([ini_config[0, i] - Ryaw*ini_config[2, i] for i in range(3)])) < 0.99*Ryaw:
                    raise Exception('Inside right sphere')

Additional 1.csv
Roll angle is  -0.5235987755982988
Ryaw is  20
Ryaw is  30
Ryaw is  40
Ryaw is  50


Exception: Inside right sphere